In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from statistics import mean
from sklearn import svm
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import LeaveOneOut
from xgboost import XGBClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import GridSearchCV

In [4]:
columns = ['alpha25', 'target']

In [5]:
df = pd.read_excel("F:\Alabama_Internship\EEG\Deepesh Code\Dataset_alpha_peak_power.xlsx")
# df = Final_df[columns]

X = df.iloc[:, 1 : -1].values
y = df.iloc[:, -1].values

X = pd.DataFrame(X)
y = pd.DataFrame(y)

Participants_count = len(df.index)//2

LR_scores = []
XGB_scores = []
KNN_scores = []
SVM_scores = []
KSVM_scores = []
NB_scores = []
DTC_scores = []
RFC_scores = []
Model_scores_list = []

In [6]:
X.head()

,0,1,2,3,4,5,6,7,8,9,...,54,55,56,57,58,59,60,61,62,63
0,1.740384e-11,7.644430e-12,5.432938e-12,1.124353e-12,8.868599e-12,4.218881e-12,1.321709e-12,4.772298e-12,2.658615e-12,5.581359e-12,...,2.319765e-11,1.356018e-11,3.682415e-12,1.188451e-11,6.500905e-12,4.675089e-12,1.969562e-11,2.237665e-11,1.599308e-11,1.116844e-11
1,1.090923e-11,8.457752e-12,6.524322e-12,3.805342e-12,9.368088e-12,6.356840e-12,4.086154e-12,6.338049e-12,6.946242e-12,7.454678e-12,...,1.051873e-11,7.299179e-12,3.607953e-12,1.235707e-11,7.796453e-12,6.553618e-12,1.996469e-11,2.152094e-11,1.481425e-11,1.352919e-11
2,2.405805e-11,1.765724e-11,1.435038e-11,3.584352e-12,2.680430e-11,1.637299e-11,8.036387e-12,2.094155e-11,7.374669e-12,1.660086e-11,...,2.211454e-11,8.955110e-12,6.435883e-12,1.314133e-11,1.178474e-11,1.159826e-11,5.905741e-11,1.465881e-10,5.326946e-11,3.102119e-11
3,8.457811e-11,2.867029e-11,2.558256e-11,1.091754e-11,5.496820e-11,3.521418e-11,1.628092e-11,5.291875e-11,4.081679e-11,7.046233e-11,...,8.639398e-11,3.249568e-11,7.697246e-12,5.347042e-11,1.404620e-11,1.318877e-11,8.956166e-11,1.140619e-10,1.038501e-10,9.709948e-11
4,1.265779e-10,6.090300e-11,5.969947e-11,3.027074e-11,1.303301e-10,9.874664e-11,3.040158e-11,1.174445e-10,1.023657e-10,1.376136e-10,...,1.249913e-10,3.288294e-11,8.048853e-12,7.501919e-11,5.139438e-12,1.580630e-11,1.458101e-10,1.641819e-10,1.731148e-10,1.732725e-10


For Leave one out cross validation (LOOCV method), drop out the pre and post info related to one participant, train the model using the remaining data. Test the model using the info which was dropped during the training stage.

In [7]:
print(Participants_count)

20


In [8]:
for i in range(Participants_count):

    X_train = X.drop(labels = [i, i + Participants_count], axis=0)
    y_train = y.drop(labels = [i, i + Participants_count], axis=0)
    X_test = X.iloc[[i, i + Participants_count],:] 
    y_test = y.iloc[[i, i + Participants_count],:] 

    if(i==0):
        X_train.to_excel("X_dataframe.xlsx")
        y_train.to_excel("y_dataframe.xlsx")

    # Feature Scaling

    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)

    y_train = y_train.values.ravel()
    y_test = y_test.values.ravel()

    # Dimensionality Reduction

    # PCA - Principle Component Analysis

    #pca = PCA(n_components = 0.95)
    #X_train = pca.fit_transform(X_train)
    #X_test = pca.transform(X_test)
    #explained_variance = pca.explained_variance_ratio_
    #print(explained_variance)

    # LDA - Linear Discriminant Analysis

    #lda = LDA(n_components = 1)
    #X_train = lda.fit_transform(X_train,y_train)
    #X_test = lda.transform(X_test)

    # ML Models Accuracy Computation

    LR_classifier = LogisticRegression(random_state = 0)
    LR_classifier.fit(X_train, y_train)
    LR_scores.append(LR_classifier.score(X_test,y_test))

    XGB_classifier = XGBClassifier()
    XGB_classifier.fit(X_train, y_train)
    XGB_scores.append(XGB_classifier.score(X_test,y_test))
    #plt.bar(range(len(XGB_classifier.feature_importances_)), XGB_classifier.feature_importances_)
    #plt.show()

    KNN_classifier = KNeighborsClassifier(n_neighbors = 3, metric = 'minkowski', p = 2)
    KNN_classifier.fit(X_train, y_train)
    KNN_scores.append(KNN_classifier.score(X_test,y_test))

    SVM_classifier = SVC(kernel = 'linear', random_state = 0)
    SVM_classifier.fit(X_train, y_train)
    SVM_scores.append(SVM_classifier.score(X_test,y_test))

    KSVM_classifier = SVC(kernel = 'rbf', random_state = 0)
    KSVM_classifier.fit(X_train, y_train)
    KSVM_scores.append(KSVM_classifier.score(X_test,y_test))

    NB_classifier = GaussianNB()
    NB_classifier.fit(X_train, y_train)
    NB_scores.append(NB_classifier.score(X_test,y_test))

    DTC_classifier = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
    DTC_classifier.fit(X_train, y_train)
    DTC_scores.append(DTC_classifier.score(X_test,y_test))

    RFC_classifier = RandomForestClassifier(random_state=0)
    RFC_classifier.fit(X_train, y_train)
    RFC_scores.append(RFC_classifier.score(X_test,y_test))

We print out the mean of the accuracies obtained through each of the ML models

In [9]:
model_str = ["Logistic Regression","XG Boost", "KNN", "SVM", "Kernel SVM", "Naive Bayes", "Decision Trees Classifier", "Random Forest Classifier"]

Model_scores_list.append(LR_scores)
Model_scores_list.append(XGB_scores)
Model_scores_list.append(KNN_scores)
Model_scores_list.append(SVM_scores)
Model_scores_list.append(KSVM_scores)
Model_scores_list.append(NB_scores)
Model_scores_list.append(DTC_scores)
Model_scores_list.append(RFC_scores)

for i in range(len(model_str)):

    print(model_str[i] + " Score : " + str(mean(Model_scores_list[i])))

Logistic Regression Score : 0.5
XG Boost Score : 0.525
KNN Score : 0.475
SVM Score : 0.5
Kernel SVM Score : 0.375
Naive Bayes Score : 0.275
Decision Trees Classifier Score : 0.4
Random Forest Classifier Score : 0.45
